In [ ]:
!pip install crewai
!pip install crewai-tools

In [ ]:
import os
from crewai import Agent,Task,Crew,Process,LLM
from crewai_tools import SerperDevTool

/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
from google.colab import userdata
gemini_key = userdata.get('gemini_key')
serper_key = userdata.get('serper_key')
os.environ['GOOGLE_API_KEY'] = gemini_key
os.environ['SERPER_API_KEY'] = serper_key


In [ ]:
search_tool = SerperDevTool()

llm = LLM(model = 'gemini/gemini-1.5-flash',
          verbose = True,
          temperature = 0.5,
          api_key = os.environ['GOOGLE_API_KEY'])

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
# Travel Researcher Agent (Finds historical sites + weather)
researcher = Agent(
    role="Travel Researcher",
    goal="Find historical sites, public transport hotels, and real-time weather for {destination}.",
    verbose=True,
    memory=True,
    backstory="You are an expert travel researcher, providing up-to-date information about history-focused trips.",
    llm=llm,
    tools=[search_tool],
    allow_delegation=True
)

# Budget Planner Agent (Ensures trip stays under 5000)
budget_planner = Agent(
    role="Budget Planner",
    goal="Find budget flights, hotels, and activities within {budget} for {destination}.",
    verbose=True,
    memory=True,
    backstory="You are a skilled budget analyst ensuring trips fit within financial constraints.",
    llm=llm,
    tools=[search_tool],
    allow_delegation=False
)

# Itinerary Planner Agent (Creates a balanced 3-day plan)
itinerary_planner = Agent(
    role="Itinerary Planner",
    goal="Create a 3-day itinerary for {destination}, ensuring all historical sites are covered under {budget}.",
    verbose=True,
    memory=True,
    backstory="You are an expert in trip planning, ensuring travelers get the best experience within their budget.",
    llm=llm,
    tools=[search_tool],
    allow_delegation=False
)

# Travel Research Task
research_task = Task(
    description="Find the best historical sites, weather forecast, and public transport hotels for {destination}.",
    expected_output="A list of top historical sites, a real-time weather update, and 3 hotel options near public transport.",
    tools=[search_tool],
    agent=researcher
)

# Budget Estimation Task
budget_task = Task(
    description="Find budget flights, hotel options, and daily food/transport costs for {destination}. Ensure total cost stays under {budget}.",
    expected_output="A full cost breakdown (flights, hotel, food, attractions) ensuring a $1500 budget is maintained.",
    tools=[search_tool],
    agent=budget_planner
)

# Itinerary Planning Task
itinerary_task = Task(
    description="Plan a 3-day itinerary for {destination}, focusing on historical sites, budget constraints, and real-time weather conditions.",
    expected_output="A detailed 3-day plan, considering weather and budget constraints, with transport recommendations.",
    tools=[search_tool],
    agent=itinerary_planner
)

# Crew Setup: All agents working together!
crew = Crew(
    agents=[researcher, budget_planner, itinerary_planner],
    tasks=[research_task, budget_task, itinerary_task],
    process=Process.sequential  # Runs tasks in sequence
)

# Run the CrewAI Trip Advisor system for London with a $1500 budget
result = crew.kickoff(inputs={'destination': 'Madhya pradesh', 'budget': '5000'})
print(result)

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Travel Researcher                                                                                       │
│                                                                                                                 │
│  Task: Find the best historical sites, weather forecast, and public transport hotels for Madhya pradesh.        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Travel Researcher                                                                                       │
│                                                                                                                 │
│  Thought: tool_code                                                                                             │
│  Thought: I need to gather information on historical sites, weather, and hotels near public transport in        │
│  Madhya Pradesh.  I'll start by searching for historical sites.                                                 │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"search_query\": \"Top historical sites in Madhya Pradesh\"}"                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"search_query\": \"cost of bus travel between Indore, Sanchi, and Khajuraho; entrance fees for Sanchi       │
│  Stupa and Khajuraho Temples\"}"                                                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  {'searchParameters': {'q': 'cost of bus travel between Indore, Sanchi, and Khajuraho; entrance fees for        │
│  Sanchi Stupa and Khajuraho Temples', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title':   │
│  'Khajuraho to Sanchi - 4 ways to travel via train, car, taxi, and plane', 'link':                              │
│  'https://www.rome2rio.com/s/Khajuraho/Sanchi-India', 'snippet': '... from Khajuraho to Sanchi costs only $14,  │
│  and the quickest way takes just 4¾ hours ... Select an option below to see step-by-step directions and to      │
│  compare ticket ...', 'position': 1}, {'title': 'THE 15 BEST Madhya Pradesh Bus Tours (with Prices) -           │
│  Tripadvisor', 'link':                                                                                          │
│  'https://www.tripadvisor.com/Attractions-g297646-Activities-c42-t174-Madhya_Pradesh.html', 'snippet':          │
│  'Authentic Khajuraho experience • Temples of Khajuraho from different prospective • *This tour can be          │
│  customized upon request according ...', 'position': 2}, {'title': 'Book Bus Tickets to Khajuraho and from      │
│  Khajuraho - redBus', 'link': 'https://www.redbus.in/buses/khajuraho-bus-tickets', 'snippet': 'Khajuraho bus    │
│  ticket booking : Compare and book bus tickets to Khajuraho. Find availability of buses from Khajuraho, best    │
│  deals and schedules at redBus.', 'position': 3}, {'title': 'List of Famous Temple Sanchi Stupa |               │
│  MyTicketsToIndia', 'link':                                                                                     │
│  'https://www.myticketstoindia.com/tourism-in-india/temples/madhya-pradesh/sanchi-stupa/', 'snippet': 'The      │
│  best time to visit the Sanchi Stupa is during the winter months, from November to February. ... buses and      │
│  cabs available from these cities to ...', 'position': 4}, {'title': 'Indore to Sanchi Bus Online Booking upto  │
│  200 Rs OFF | MakeMyTrip', 'link':                                                                              │
│  'https://www.makemytrip.com/bus-tickets/indore-sanchi-bus-ticket-booking.html', 'snippet': 'Bus fare starts    │
│  from 150-200 INR and may vary according to the type of bus used and the total distance traveled. You can       │
│  avail all the bus tickets from “make my ...', 'position': 5}, {'title': 'Itineraries - Packages | Madhya       │
│  Pradesh Tourism', 'link': 'https://www....                                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Itinerary Planner                                                                                       │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **Madhya Pradesh 3-Day Historical Sites Itinerary (Budget: ₹5000)**                                            │
│                                                                                                                 │
│  This itinerary prioritizes Sanchi and Khajuraho, focusing on historical sites and minimizing travel costs.     │
│  The budget is extremely tight, and flexibility is key.  Prices are estimates and may vary.  Booking in         │
│  advance is crucial for better deals.                                                                           │
│                                                                                                                 │
│  **Day 1: Sanchi**                                                                                              │
│                                                                                                                 │
│  * **Morning (8:00 AM):** Depart from Indore by bus to Sanchi.  (Estimated cost: ₹150-200, travel time: ~3-4    │
│  hours). Check bus schedules and book in advance for better prices.                                             │
│  * **Afternoon (12:00 PM):** Arrive in Sanchi. Visit the Sanchi Stupa (UNESCO World Heritage Site). Allow       │
│  ample time for exploration. (Estimated entrance fee: ₹50-100). Have lunch at a local eatery (budget: ₹150).    │
│  * **Late Afternoon (4:00 PM):** Explore the surrounding area of Sanchi, perhaps visiting the Sanchi Museum.    │
│  * **Evening (6:00 PM):** Take a bus to a budget-friendly hotel near Khajuraho. (Estimated cost: ₹200-300,      │
│  travel time: ~6-8 hours.  This may require an overnight bus).                                                  │
│                                                                                                                 │
│  **Day 2: Khajuraho**                                                                                           │
│                                                                                                                 │
│  * **Morning (8:00 AM):** Arrive in Khajuraho. Check into your pre-booked budget hotel near the railway         │
│  station (budget: ₹300).  Have breakfast (budget: ₹100).                                                        │
│  * **Morning/Afternoon (9:00 AM - 4:00 PM):** Visit the Khajuraho Temples (UNESCO World Heritage Site).         │
│  Explore the Western and Eastern temple groups.  Allow ample time for this significant site (Estimated          │
│  entrance fee: ₹500). Have lunch at a local eatery (budget: ₹150).                                              │
│  * **Evening (6:00 PM):** Relax, explore the local market, and enjoy dinner (budget: ₹150).                     │
│                                                                                                                 │
│  **Day 3: Return to Indore**                                                                                    │
│                                                                                                                 │
│  * **Morning (8:00 AM):** Have breakfast (budget: ₹100).                                                        │
│  * **Morning (9:00 AM):** Take a bus from Khajuraho to Indore. (Estimated cost: ₹300-400, travel time: ~8-10    │
│  hours.  This will likely require an overnight bus).   

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


**Madhya Pradesh 3-Day Historical Sites Itinerary (Budget: ₹5000)**

This itinerary prioritizes Sanchi and Khajuraho, focusing on historical sites and minimizing travel costs.  The budget is extremely tight, and flexibility is key.  Prices are estimates and may vary.  Booking in advance is crucial for better deals.

**Day 1: Sanchi**

* **Morning (8:00 AM):** Depart from Indore by bus to Sanchi.  (Estimated cost: ₹150-200, travel time: ~3-4 hours). Check bus schedules and book in advance for better prices.
* **Afternoon (12:00 PM):** Arrive in Sanchi. Visit the Sanchi Stupa (UNESCO World Heritage Site). Allow ample time for exploration. (Estimated entrance fee: ₹50-100). Have lunch at a local eatery (budget: ₹150).
* **Late Afternoon (4:00 PM):** Explore the surrounding area of Sanchi, perhaps visiting the Sanchi Museum.
* **Evening (6:00 PM):** Take a bus to a budget-friendly hotel near Khajuraho. (Estimated cost: ₹200-300, travel time: ~6-8 hours.  This may require an overnight bus).